# We aim to use ML to make content based recommendation for new users.



In [1]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

# read in the csv files
portfolio = pd.read_csv('portfolio.csv')
profile = pd.read_csv('profile.csv')
transcript = pd.read_csv('transcript.csv')

- Removing unnamed column

In [2]:
transcript.drop('Unnamed: 0', axis=1, inplace=True)
profile.drop('Unnamed: 0', axis=1, inplace=True)
portfolio.drop('Unnamed: 0', axis=1, inplace=True)

In [3]:
portfolio.head(10)

reward                              channels  difficulty  duration  \
0      10         ['email', 'mobile', 'social']          10         7   
1      10  ['web', 'email', 'mobile', 'social']          10         5   
2       0            ['web', 'email', 'mobile']           0         4   
3       5            ['web', 'email', 'mobile']           5         7   
4       5                      ['web', 'email']          20        10   
5       3  ['web', 'email', 'mobile', 'social']           7         7   
6       2  ['web', 'email', 'mobile', 'social']          10        10   
7       0         ['email', 'mobile', 'social']           0         3   
8       5  ['web', 'email', 'mobile', 'social']           5         5   
9       2            ['web', 'email', 'mobile']          10         7   

      offer_type                                id  
0           bogo  ae264e3637204a6fb9bb56bc8210ddfd  
1           bogo  4d5c57ea9a6940dd891ad53e9dbe8da0  
2  informational  3f207df678b143eea3cee63160fa8bed  
3           bogo  9b98b8c7a33c4b65b9aebfe6a799e6d9  
4       discount  0b1e1539f2cc45b7b9fa7c272da2e1d7  
5       discount  2298d6c36e964ae4a3e7e9706d1fb8c2  
6       discount  fafdcd668e3743c1bb461111dcafc2a4  
7  informational  5a8bc65990b245e5a138643cd4eb9837  
8           bogo  f19421c1d4aa40978ebb69ca19b0e20d  
9       discount  2906b810c7d4411798c6938adc9daaa5

In [4]:
portfolio.shape

(10, 6)

Portfolio data has 10 rows for 10 different offers. There are three types of offers : **BOGO, Discount and Informational**. <br>
To redeem the offers one has to spend 0, 5, 7, 10 or 20 dollars (difficulty column).

#### Combine profile and transcript df

1. Change **became_member_on** column to datetime column
2. Join datasets on person/id column

In [5]:
profile_data = profile.copy()
transcript_data = transcript.copy()

In [6]:
# change became_member_on to datetime format and drop the column
profile_data['date'] = pd.to_datetime(profile_data["became_member_on"],format='%Y%m%d')
profile_data.drop('became_member_on',axis=1,inplace=True)

In [7]:
# join on person/id column
new_data = transcript_data.join(profile_data.set_index('id'), on = 'person')

In [8]:
new_data.head()

person           event  \
0  78afa995795e4d85b5d9ceeca43f5fef  offer received   
1  a03223e636434f42ac4c3df47e8bac43  offer received   
2  e2127556f4f64592b11af22de27a7932  offer received   
3  8ec6ce2a7e7949b1bf142def7d0e0586  offer received   
4  68617ca6246f4fbc85e91a2a49552598  offer received   

                                              value  time gender  age  \
0  {'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'}     0      F   75   
1  {'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'}     0    NaN  118   
2  {'offer id': '2906b810c7d4411798c6938adc9daaa5'}     0      M   68   
3  {'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'}     0    NaN  118   
4  {'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'}     0    NaN  118   

     income       date  
0  100000.0 2017-05-09  
1       NaN 2017-08-04  
2   70000.0 2018-04-26  
3       NaN 2017-09-25  
4       NaN 2017-10-02

Extract offer id and transaction amount and replace the value column

In [9]:
#Converting string to dict
import json
def string_to_dict(dict_string):
    # Convert to proper json format
    dict_string = dict_string.replace("'", '"').replace('u"', '"')
    return json.loads(dict_string)

new_data.value = new_data.value.apply(string_to_dict)

In [10]:
# extract offer id and transaction amount and replace the value column

new_data['value'] = new_data.apply(lambda x : list(x['value'].values())[0],axis = 1)

In [11]:
new_data.head()

person           event  \
0  78afa995795e4d85b5d9ceeca43f5fef  offer received   
1  a03223e636434f42ac4c3df47e8bac43  offer received   
2  e2127556f4f64592b11af22de27a7932  offer received   
3  8ec6ce2a7e7949b1bf142def7d0e0586  offer received   
4  68617ca6246f4fbc85e91a2a49552598  offer received   

                              value  time gender  age    income       date  
0  9b98b8c7a33c4b65b9aebfe6a799e6d9     0      F   75  100000.0 2017-05-09  
1  0b1e1539f2cc45b7b9fa7c272da2e1d7     0    NaN  118       NaN 2017-08-04  
2  2906b810c7d4411798c6938adc9daaa5     0      M   68   70000.0 2018-04-26  
3  fafdcd668e3743c1bb461111dcafc2a4     0    NaN  118       NaN 2017-09-25  
4  4d5c57ea9a6940dd891ad53e9dbe8da0     0    NaN  118       NaN 2017-10-02

In [12]:
#Change gender from None to unknown
new_data['gender'] = new_data['gender'].apply(lambda x: 'Unknown' if x == None else x)

Since we have **offer id** and **transaction amount** in the same column, we can split the dataset into offer data and transaction data for further analysis

In [13]:
offer_data = new_data[new_data['event'] != 'transaction']
transaction_data = new_data[new_data['event'] == 'transaction']
#Convert value amount to float
transaction_data = transaction_data.copy() # to avoid settingwithcopy warning
transaction_data.loc[:,'value'] = transaction_data.loc[:,'value'].astype(float)

#### Combine portfolio data with offer data

This will help us find out what type of offers are more popular among consumers

In [14]:
offer_data = offer_data.join(portfolio.set_index('id'), on = 'value')

## Building a Recommendation System

In [15]:
#View offers completed by each person
offer_data[offer_data['event'] == 'offer completed'].groupby('person')['event'].count()

person
0009655768c64bdeb2e877511632db8f    3
0011e0d4e6b944f998e987f904e8c1e5    3
0020c2b971eb4e9188eac86d93036a77    3
0020ccbbb6d84e358d3414a3ff76cffd    3
003d66b6608740288d6cc97a6903f4f0    3
                                   ..
fff29fb549084123bd046dbc5ceb4faa    6
fff3ba4757bd42088c044ca26d73817a    3
fff7576017104bcc8677a8d63322b5e1    3
fffad4f4828548d1b5583907f2e9906b    3
ffff82501cea40309d5fdd7edcca4a07    6
Name: event, Length: 12774, dtype: int64



To train an ML model:

- We want to predict the expenditure a person will make on starbucks offers , given the demograpic information.
- In this data, 'person','event','time', and 'date' column is not necessary.
- We will groupby the data by 'person' and try to predict the mean value.

In [16]:
# Data preparation

def data_load(df = transaction_data):
    '''
    INPUT:
    df - (DataFrame) pandas dataframe of transaction data
    
    OUTPUT:
    X: Features (gender,age,income columns)
    y: target variable(value column)
    non_demo_data: mean value of the non demograpic data
    Description:
    Drop 'event','time','date' columns from the dataframe, groupby the data 
    by 'person' column.
    Take mean of value column and max/min of the other columns 
    
    '''
    new_df = df.drop(['event','time','date' ],axis = 1)
    non_demo_value = np.round(new_df[new_df['age'] == 118]['value'].median(),decimals = 2)
    new_df = new_df[new_df['age'] != 118] # remove outliers
    # groupby person
    new_df = new_df.groupby('person',as_index = False) 

    X = new_df[['gender','income','age']].max()
    X.drop('person',axis=1,inplace=True) #Drop person column
    y = np.round(new_df['value'].median(),decimals = 2)
    y.drop('person',axis=1,inplace=True) #Drop person column
    return X,y,non_demo_value

In [17]:
X_data,y_data,non_demo_val = data_load()

In [18]:
print(X_data.head())
print(y_data.head())


  gender   income  age
0      M  72000.0   33
1      O  57000.0   40
2      F  90000.0   59
3      F  60000.0   24
4      F  73000.0   26
   value
0  13.84
1  13.49
2  24.35
3  12.76
4   2.27


#### Data preprocessing using SKLearn



**Preprocessing Categorical Attribute**

In [19]:
#Importing libraries
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder,StandardScaler,FunctionTransformer,PolynomialFeatures
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn import svm
import xgboost as xgb

cat_encoder = OneHotEncoder()
data_cat_1hot = cat_encoder.fit_transform(X_data[['gender']])

**Imputation of numerical values**

In [20]:


def numerical_scl(df):
    '''
    INPUT:
    df - (DataFrame) pandas dataframe 
    
    OUTPUT:
    num_attr : numerical attributes
    num_pipeline: numerical pipeline object
    Description:
    Using Scikit learn to impute NaN values, and standardize     
    '''
    nums = ['float64','int64']
    
    num_attr = df.select_dtypes(include = nums)
    
    num_pipeline = Pipeline([
        ('log',FunctionTransformer(np.log)),
        ('std_scaler', StandardScaler()),
        ('poly',PolynomialFeatures(3,interaction_only=False,include_bias = False))
        
    ])
    
    return num_attr,num_pipeline
    

#### Transforming and scaling numerical data with encoding categorical data

In [21]:
def full_data_pipeline(df):
    """
    INPUT:
    df - (DataFrame) pandas dataframe 
    
    OUTPUT:
    full_data_pipeline : final prepared data pipeline to feed into the ML model
    Description:
    full data transformation pipeline for both numerical and categorical data
    """
    cat_attr = ["gender"]
    
    num_attr,num_pipeline = numerical_scl(df)
    full_data_pipeline = ColumnTransformer([
        ('num',num_pipeline,list(num_attr)),
        ('cat',OneHotEncoder(),cat_attr),
        ])
    
    
    
    return full_data_pipeline
    

## Train an ML model

Since this is a regression problem, we can use various regression methods to find the best model.

We have used:
- Linear Regression
- Random Forest Regression
- XGBoost regressor

In [22]:
pipe = full_data_pipeline(X_data)
X_data_mod = pipe.fit_transform(X_data)

- Test Train Split

In [23]:
xtrain,xtest,ytrain,ytest = train_test_split(X_data_mod, y_data.values.clip(0,25).ravel(), test_size=0.2, random_state=42)

- Initialize Models

In [24]:
rf_reg = RandomForestRegressor(random_state=42)
lin_reg = LinearRegression()
xgb_reg = xgb.XGBRegressor(base_score=0.75, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.01, max_delta_step=0, max_depth=5,
             min_child_weight=4,n_estimators=500, n_jobs=8, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

- Fitting data to the Models

In [25]:
rf_reg.fit(xtrain,ytrain)
lin_reg.fit(xtrain,ytrain)
xgb_reg.fit(xtrain,ytrain)

XGBRegressor(base_score=0.75, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
             grow_policy='depthwise', importance_type='gain',
             interaction_constraints='', learning_rate=0.01, max_bin=256,
             max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
             max_depth=5, max_leaves=0, min_child_weight=4, missing=nan,
             monotone_constraints='()', n_estimators=500, n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=0, ...)

### Calculating Scores
- Training set

In [26]:
print('R_2 score on training set for randomforest regressor:',rf_reg.score(xtrain,ytrain))
print('R_2 score on training set for linear regressor:',lin_reg.score(xtrain,ytrain))
print('R_2 score on training set for XGBoost regressor:',xgb_reg.score(xtrain,ytrain))

R_2 score on training set for randomforest regressor: 0.8488589883726012
R_2 score on training set for linear regressor: 0.6801470940270071
R_2 score on training set for XGBoost regressor: 0.7294419335717145


- Testing set

In [27]:
print('R_2 score on testing set for randomforest regressor:',rf_reg.score(xtest,ytest))
print('R_2 score on testing set for linear regressor:',lin_reg.score(xtest,ytest))
print('R_2 score on testing set for XGBoost regressor:',xgb_reg.score(xtest,ytest))

R_2 score on testing set for randomforest regressor: 0.6233133530158258
R_2 score on testing set for linear regressor: 0.6763578294368009
R_2 score on testing set for XGBoost regressor: 0.7011693474487255


#### The best model for prediction of expense is xgboost with R_Squared value of 70.11

- Performing Cross validation

In [28]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(xgb_reg, X_data_mod, y_data.values.clip(0,25).ravel(), cv=5)
scores

array([0.71919124, 0.72262426, 0.68972657, 0.69853782, 0.6897438 ])

In [29]:
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

0.70 accuracy with a standard deviation of 0.01


#### Hyperparameter tuning for XG boost

In [30]:
booster = ['gbtree','gblinear']
base_score = [0.25,0.5,0.75,1]
n_estimators = [100,500,900,1100,1500]
max_depth = [2,3,5,10,15]
learning_rate = [0.05,0.1,0.15,0.20]
min_child_weight = [1,2,3,4,5]

hyperparameter_grid = {
    'n_estimators': n_estimators,
    'max_depth' : max_depth,
    'learning_rate' : learning_rate,
    'min_child_weight' : min_child_weight,
    'booster' : booster,
    'base_score': base_score
}

In [31]:
from sklearn.model_selection import RandomizedSearchCV
random_cv = RandomizedSearchCV(estimator = xgb_reg,
                              param_distributions = hyperparameter_grid,
                              cv = 5, n_iter = 20,
                              scoring = 'neg_mean_absolute_error',n_jobs = -1,
                              verbose = 5,
                              return_train_score = True,
                              random_state = 42)

In [32]:
random_cv.fit(xtrain,ytrain)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


RandomizedSearchCV(cv=5,
                   estimator=XGBRegressor(base_score=0.75, booster='gbtree',
                                          callbacks=None, colsample_bylevel=1,
                                          colsample_bynode=1,
                                          colsample_bytree=1,
                                          early_stopping_rounds=None,
                                          enable_categorical=False,
                                          eval_metric=None, feature_types=None,
                                          gamma=0, gpu_id=-1,
                                          grow_policy='depthwise',
                                          importance_type='gain',
                                          interaction_constraints='',
                                          learning_rate=0.0...
                                          num_parallel_tree=1, predictor='auto',
                                          random_state=0, ...),
    

In [33]:

XGboost_best=random_cv.best_estimator_

In [34]:
from sklearn.metrics import mean_squared_error

mean_squared_error(ytest, xgb_reg.predict(xtest), squared=False)

4.633044291213493

In [35]:
print('R_2 score on testing set for XGBoost regressor:',XGboost_best.score(xtest,ytest))

R_2 score on testing set for XGBoost regressor: 0.699624871255685


In [36]:
def predict_expense(demographics,model):
    if type(demographics) == dict:
        df= pd.DataFrame(demographics)
    else:
        df = demographics
    data_pipe = full_data_pipeline(X_data)
    prep_data = data_pipe.fit_transform(df)
    y_pred = model.predict(prep_data)
    
    return np.round(y_pred,decimals=2)

- Predicting using model

In [37]:
person_demo = {
    'gender' : ['M','F','O','O'],
    'income' : [62345.0,72045.0,80130.0,678.0],
    'age' : [30,29,30,41]
}

predict_expense(person_demo,xgb_reg)

array([10.51, 17.79, 19.13,  4.35], dtype=float32)

### The last step is to create rules for extending offers to new users
- We can send out a **Informational offer or No offer** as a person with no demographic information is predicted to spend only about $1.71. 
- Send out a **Informational offer or no offer** if a user is anticipated to spend less than $3 or more than $22. 
- Send out the **$5 discount offers** if a consumer is anticipated to spend between $3 and $5.
- Send out the **$7 discount offer** if a person is anticipated to spend between $5 and $7. 
- Send the **$10 vouchers** if a consumer is anticipated to spend between $7 and $10.
-  Finally, if a user is anticipated to spend between $10 and $22, send the **$20 coupon**.